In [1]:
import json
with open('/Users/marya/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [3]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info
            

In [4]:
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'genres_.csv.gz',
 'Project 3-EDA-TMDB.ipynb',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_genres.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [6]:
YEARS_TO_GET = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

In [ ]:
from tqdm.notebook import tqdm_notebook
import pandas as pd
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
        JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
        file_exists = os.path.isfile(JSON_FILE)
        if file_exists == False:
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)

        basics = pd.read_csv("Data/title_basics.csv.gz")
        df = basics.loc[ basics['startYear']==YEAR].copy()
        movie_ids = df['tconst'].copy()
        previous_df = pd.read_json(JSON_FILE)
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

        def write_json(new_data, filename):   
            with open(filename,'r+') as file:
                file_data = json.load(file)
                if (type(new_data) == list) & (type(file_data) == list):
                    file_data.extend(new_data)
                else:
                     file_data.append(new_data)
                file.seek(0)
                json.dump(file_data, file)

        for movie_id in tqdm_notebook(movie_ids_to_get,
                                              desc=f'Movies from {YEAR}',
                                              position=1,
                                              leave=True):
                try:
                        temp = get_movie_with_rating(movie_id)
                        write_json(temp,JSON_FILE)
                        time.sleep(0.02)

                except Exception as e:
                        continue

        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/6065 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/6829 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/7366 [00:00<?, ?it/s]